<a href="https://colab.research.google.com/github/yashkumar181/Algo-Lab-241B292/blob/main/earth_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filter Landsat 8 to a date range


Write Earth Engine Python code to filter a Landsat 8 collection to a date range from December 1, 2020 to March 1, 2021 and display the median composite on the map.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

# Define the start and end dates for the given date range
start_date = '2020-12-01'
end_date = '2021-03-01'

# Filter the Landsat 8 collection to the given date range.
# The end date is exclusive.
image_collection = ee.ImageCollection(
    'LANDSAT/LC08/C02/T1').filterDate(start_date, end_date)

# Calculate the median composite
median_composite = image_collection.median()

# Display the median composite on the map
m.add_layer(
    median_composite,
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max':30000},
    'Median Composite')
m

# Show NDVI from Sentinel-2 SR over Milan


Write Earth Engine Python code to show NDVI from Sentinel-2 SR images over Milan, Italy.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

x = 9.1895
y = 45.4660

# Define the start and end dates for the Sentinel-2 image collection
start_date = '2019-06-01'
end_date = '2019-06-15'

# Load the Sentinel-2 SR image collection
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterDate(start_date, end_date)
              .filterBounds(ee.Geometry.Point(x, y)).mosaic())


# Calculate NDVI
ndvi = image.normalizedDifference(['B8', 'B4'])

# Define visualization parameters
vis_params = {
  'min': -1,
  'max': 1,
  'palette': ['red', 'white', 'green']
}


# Add layer to the map
m.add_layer(ndvi, vis_params, 'NDVI')
# Set the center of the map to Milan, Italy
m.set_center(x, y, 13)
m

# Display SRTM DEM over Seattle


Write Earth Engine Python code to display the elevation data from the SRTM DEM over Seattle, Washington.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

# Load SRTM DEM
srtm = ee.Image('USGS/SRTMGL1_003')

# Select elevation band
elevation = srtm.select('elevation')

# Define visualization parameters
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000'],
}

# Add layer to the map
m.set_center(-122.3321, 47.6062, 10)
m.add_layer(elevation, vis_params, 'Elevation')
m

# Calculate mean NDVI on the 10km buffer around Paris using Sentinel-2


Write Earth Engine Python code to create a 10km buffer around a point representing Paris, France, and calculate the mean NDVI within the buffer zone from a Sentinel-2 image composite.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

# Define coordinates for Paris
x = 2.3522
y = 48.8566

# Create a point geometry for Paris
paris_point = ee.Geometry.Point([x, y])

# Create a 10km buffer around Paris
buffer = paris_point.buffer(10000)

# Change the dates as necessary.
start_date = '2023-07-01'
end_date = '2023-07-15'

# Load a Sentinel-2 composite. It's also possible to use median() or other ways
# to produce a composite.
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterDate(start_date, end_date)
              .mosaic())

# Calculate NDVI
ndvi = image.normalizedDifference(['B8', 'B4'])

# Calculate mean NDVI within the buffer
mean_ndvi = ndvi.reduceRegion(
  reducer=ee.Reducer.mean(),
  geometry=buffer,
  scale=100 # Scale in meters, might need to be changed.
)

# Print the mean NDVI
print('Mean NDVI:', mean_ndvi.get('nd').getInfo())

# Display the buffer and NDVI on the map
m.set_center(x, y, 10)
m.add_layer(buffer, {}, 'Buffer')
m.add_layer(
    ndvi,
    {'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}, 'NDVI')
m

# Mask out Sentinel-2 pixels above 100 m using SRTM


Write Earth Engine Python code to mask out pixels above 100 m from a Sentinel-2 image using the SRTM dataset.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

x = -122.4194
y = 37.7749

start_date = '2022-06-01'
end_date = '2022-06-15'

image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterDate(start_date, end_date)
              .filterBounds(ee.Geometry.Point(x, y)).mosaic())

srtm = ee.Image('USGS/SRTMGL1_003')

elevation = srtm.select('elevation')

elevation_threshold = 100

masked_image = image.updateMask(elevation.lt(elevation_threshold))

m.add_layer(
    elevation,
    {'min': 0, 'max': 1000, 'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000']},
    'Elevation')

m.add_layer(
    masked_image,
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1500}, 'Masked Image')
m.set_center(x, y, 10)
m

# Show NDVI over Rio using Landsat 8

Write Earth Engine Python code to show the Normalized Difference Vegetation Index (NDVI) for a collection of Landsat 8 images over Rio de Janeiro, Brazil

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

# Define the area of interest
rio = ee.Geometry.Point([-43.1729, -22.9068])

# Load the Landsat 8 collection
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(rio) \
    .filterDate('2020-01-01', '2020-03-01') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Function to calculate NDVI
def calculateNDVI(img):
    ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return img.addBands(ndvi)

# Map the NDVI function over the image collection
landsatNDVI = landsat8.map(calculateNDVI)

# Create a multi-band median composite
medianComposite = landsatNDVI.median()

# Display the NDVI composite on the map
ndviParams = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}
m.centerObject(rio, 10)
m.add_layer(medianComposite.select('NDVI'), ndviParams, 'Landsat 8 NDVI')
m

# Apply QA_PIXEL cloud mask to Landsat 8


Write Earth Engine Python code to apply the QA_PIXEL cloud mask to a Landsat 8 image. Display the original and masked images on the map.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

# Load a Landsat 8 image
image = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
              .filter(ee.Filter.greaterThan('CLOUD_COVER', 30))
              .filter(ee.Filter.lessThan('CLOUD_COVER', 60)).first())

# Define a function to mask clouds using the QA_PIXEL band
def maskL8sr(image):
  # Bit 2 is cirrus, bit 3 is cloud , bit 4 is cloud shadow
  cloudShadowBitMask = (1 << 2) | (1 << 3) | (1 << 4)
  # Get the pixel QA band
  qa = image.select('QA_PIXEL')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  return image.updateMask(mask)

# Apply the cloud mask
masked_image = maskL8sr(image)

# Define visualization parameters
vis_params = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 50000
}

# Center the map on the image
m.centerObject(image, 7)

# Add both original and masked images to the map
m.add_layer(image, vis_params, 'Original Image')
m.add_layer(masked_image, vis_params, 'Masked Image')
m

# Show average yearly temperatures over California


Write Earth Engine Python code to show a map of the average temperature over a year for a region in California.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

# Define the region of interest in California
california_region = ee.Geometry.Rectangle([-124.48, 32.53, -114.13, 42.01])

# Load ERA5 daily temperature data
era5_daily = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")

# Filter to a specific year (example: 2022)
era5_2022 = era5_daily.filterDate('2022-01-01', '2022-12-31')

# Select the temperature band
temp = era5_2022.select('temperature_2m')

# Calculate the annual average temperature
annual_avg_temp = temp.mean().clip(california_region)

# Define visualization parameters
vis_params = {
  'min': 273.15,  # 0 degrees Celsius in Kelvin
  'max': 303.15,  # 30 degrees Celsius in Kelvin
  'palette': ['blue', 'green', 'yellow', 'red']
}

# Center the map on the region and add the average temperature layer
m.set_center(-121.5, 38.5, 6)
m.add_layer(annual_avg_temp, vis_params, 'Average Temperature (2022)')
m

# Show a bar chart of MODIS land cover classes


Write Earth Engine Python code to show a bar chart of the area of each MODIS land cover class within a given country.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

import matplotlib.pyplot as plt

# Data import and filtering
modis_collection = ee.ImageCollection(
    'MODIS/061/MCD12Q1').filterDate('2019-01-01', '2020-01-01')

country = ee.FeatureCollection('WM/geoLab/geoBoundaries/600/ADM0').filter(
    ee.Filter.equals('shapeName', 'Suriname'))

land_cover_image = modis_collection.select('LC_Type1').first().clip(country)

land_cover_classes = ee.List.sequence(1, 17) # Assuming 17 land cover classes

def calculate_area(class_value):
  class_mask = land_cover_image.eq(ee.Number(class_value).toInt())
  class_area = class_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=country,
    scale=500,
    maxPixels=1e13
    ).get('LC_Type1')
  return ee.Feature(
    None, {'class_value': class_value, 'class_area': class_area})

area_results = ee.FeatureCollection(land_cover_classes.map(calculate_area))
df = ee.data.computeFeatures(
{'expression': area_results, 'fileFormat': 'PANDAS_DATAFRAME'})

# Create bar chart using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(df['class_value'], df['class_area'])
plt.xlabel('Land Cover Class')
plt.ylabel('Area (square meters)')
plt.title('Area of MODIS Land Cover Classes in Suriname (2019)')

# Set x-ticks to be integers and show all classes
plt.xticks(df['class_value'], rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Show a line chart of CHIRPS precipitation


Write Earth Engine Python code to create a line chart showing the time series of precipitation data for a specific location in California using the CHIRPS dataset.

In [ ]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()

import pandas as pd
import matplotlib.pyplot as plt

# Define a point of interest in California (example: San Francisco)
point = ee.Geometry.Point([-122.4194, 37.7749])

# Load CHIRPS precipitation data
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

# Filter data for a specific time period (example: 2022)
chirps_filtered = chirps.filterDate('2022-01-01', '2022-12-31')

# Extract time series data at the point using the 5000m scale
time_series = ee.data.computeValue(chirps_filtered.getRegion(point, 5000))

# Convert to pandas DataFrame
df = pd.DataFrame(time_series[1:], columns=time_series[0])

# Convert date column to datetime objects
df['datetime'] = pd.to_datetime(df['time'], unit='ms')

# Set datetime as index
df = df.set_index('datetime')

# Select the precipitation band ('precipitation')
precipitation = df['precipitation']

# Plot the time series using matplotlib
plt.figure(figsize=(12, 6))
plt.plot(precipitation)
plt.title('CHIRPS Precipitation Time Series (San Francisco, 2022)')
plt.xlabel('Date')
plt.ylabel('Precipitation (mm/pentad)')
plt.grid(True)
plt.show()